## M5 Forecasting Accuracy - Modeling Experiment

## Loading Packages

In [1]:
## Import Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from downcast import reduce
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
import pickle
from sklearn import linear_model
import lightgbm
from lightgbm import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor

## Loading Files

In [13]:
## Load files
prices = pd.read_csv("sell_prices.csv")

calendar = pd.read_csv("calendar.csv")

sales = pd.read_csv("sales_train_evaluation.csv")

## Feature Engineering

In [14]:
## Fill for nans
calendar.fillna('No_event',inplace=True)
calendar.head(5)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,No_event,No_event,No_event,No_event,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,No_event,No_event,No_event,No_event,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,No_event,No_event,No_event,No_event,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,No_event,No_event,No_event,No_event,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,No_event,No_event,No_event,No_event,1,0,1


In [15]:
## Reduce the size of the file
sales= reduce(sales)
prices= reduce(prices)
calendar= reduce(calendar)

##
As we identified sales are high from Jan to May, we will mark them as high sales month

In [16]:
f=lambda x: 1 if x==2 or x==3 or x==4 or x==5 else 0
calendar['is_high_sale_months']=calendar['month'].map(f) 
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,is_high_sale_months
0,2011-01-29,11101,Saturday,1,1,2011,d_1,No_event,No_event,No_event,No_event,0,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,No_event,No_event,No_event,No_event,0,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,No_event,No_event,No_event,No_event,0,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,No_event,No_event,No_event,No_event,1,1,0,1
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,No_event,No_event,No_event,No_event,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,No_event,No_event,No_event,No_event,0,1,1,0
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,No_event,No_event,No_event,No_event,0,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,No_event,No_event,No_event,No_event,0,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,No_event,No_event,No_event,No_event,0,0,0,0


##
Sales are highest on weekends, i.e Friday, saturday and Sunday, hence they would be marked as high sales day

In [17]:
f=lambda x: 1 if x==7 or x==1 or x==2 else 0
calendar['is_high_sale_day']=calendar['wday'].map(f) 
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,is_high_sale_months,is_high_sale_day
0,2011-01-29,11101,Saturday,1,1,2011,d_1,No_event,No_event,No_event,No_event,0,0,0,0,1
1,2011-01-30,11101,Sunday,2,1,2011,d_2,No_event,No_event,No_event,No_event,0,0,0,0,1
2,2011-01-31,11101,Monday,3,1,2011,d_3,No_event,No_event,No_event,No_event,0,0,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,No_event,No_event,No_event,No_event,1,1,0,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,No_event,No_event,No_event,No_event,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,No_event,No_event,No_event,No_event,0,1,1,0,0
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,No_event,No_event,No_event,No_event,0,0,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,No_event,No_event,No_event,No_event,0,0,0,0,1
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,No_event,No_event,No_event,No_event,0,0,0,0,1


##
Christmas is a no sales day, probably because the stores are closed

In [18]:
f=lambda x: 1 if x=='Christmas' else 0
calendar['is_christmas']=calendar['event_name_1'].map(f)
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,is_high_sale_months,is_high_sale_day,is_christmas
0,2011-01-29,11101,Saturday,1,1,2011,d_1,No_event,No_event,No_event,No_event,0,0,0,0,1,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,No_event,No_event,No_event,No_event,0,0,0,0,1,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,No_event,No_event,No_event,No_event,0,0,0,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,No_event,No_event,No_event,No_event,1,1,0,1,0,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,No_event,No_event,No_event,No_event,1,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,No_event,No_event,No_event,No_event,0,1,1,0,0,0
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,No_event,No_event,No_event,No_event,0,0,0,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,No_event,No_event,No_event,No_event,0,0,0,0,1,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,No_event,No_event,No_event,No_event,0,0,0,0,1,0


In [19]:
df_sales=pd.melt(sales, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='sold')     

In [20]:
df_sales

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
...,...,...,...,...,...,...,...,...
59181085,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1941,1
59181086,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1941,0
59181087,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1941,2
59181088,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1941,0


In [21]:
df_final= pd.merge(df_sales, calendar, on='d', how='left')

In [22]:
df_final= pd.merge(df_final, prices, on=['store_id','item_id','wm_yr_wk'], how='left') 

## 1. Introducing Lag features
Our data lacks periodicity in terms of month/ year, but there is periodicity in weeks, jence we will introduce lags on weekly basis
lag 1:means how many products have been sold one day before.
lag 7:means how many products have been sold seven day before.

In [23]:
import tqdm
lags=[1,7,14,21,28,35,42,49,56,63,70]
for i in tqdm.tqdm(lags):
    df_final['lag_'+str(i)]=df_final.groupby(['id'])['sold'].shift(i)

100%|██████████| 11/11 [00:12<00:00,  1.16s/it]


In [24]:
df_final.head(5)

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,date,wm_yr_wk,...,lag_7,lag_14,lag_21,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# import pickle
# with open(r'a_before_rolling.pkl','wb') as f:
#     pickle.dump(df_final, f)

In [26]:
# import shutil
# original=r'C:\Users\Owner\Documents\Model 1\m5-forecasting-accuracy\a_before_rolling.pkl'
#target=r'C:\Users\Owner\Documents\Model 1\m5-forecasting-accuracy\a_before_rolling.pkl'
#shutil.copyfile(original, target)

## 2. Rolling Window
This is another technique to capture perriodicity.
in this technique we are going to apply some statistical techniques on window.here window mean last some days
example if i say window=7 and statistical techniques=mean, then we are going to take last 7 day and apply mean on them the result will be your feature. we can apply median , min max .. any thing.

In [27]:
window=[7,14,28,35,42]
for i in tqdm.tqdm(window):
    df_final['rolling_mean_'+str(i)]=df_final.groupby(['id'])['sold'].transform(lambda s: s.rolling(i).mean())

100%|██████████| 5/5 [02:14<00:00, 26.97s/it]


In [28]:
df_final.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'sold', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'is_high_sale_months',
       'is_high_sale_day', 'is_christmas', 'sell_price', 'lag_1', 'lag_7',
       'lag_14', 'lag_21', 'lag_28', 'lag_35', 'lag_42', 'lag_49', 'lag_56',
       'lag_63', 'lag_70', 'rolling_mean_7', 'rolling_mean_14',
       'rolling_mean_28', 'rolling_mean_35', 'rolling_mean_42'],
      dtype='object')

In [29]:
window=['rolling_mean_7','rolling_mean_14', 'rolling_mean_7' ,'rolling_mean_28', 'rolling_mean_35','rolling_mean_42']
for i in window:
    df_final[i]=df_final[i].fillna(0)

In [30]:
df_final=reduce(df_final)

## 3. Expanding Mean
Expanding window is same as rolling window only difference is that here window size is not constant its dynamic.

In [31]:
# with open(r'd_before_rolling.pkl','wb') as f:
#     pickle.dump(df_final, f)

In [32]:
df_final['expanding_mean']=df_final.groupby(['id'])['sold'].transform(lambda s: s.expanding().mean())

In [33]:
df_final['expanding_mean']=df_final['expanding_mean'].fillna(0)

In [34]:
df_final=reduce(df_final)

In [35]:
df_final.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,date,wm_yr_wk,...,lag_49,lag_56,lag_63,lag_70,rolling_mean_7,rolling_mean_14,rolling_mean_28,rolling_mean_35,rolling_mean_42,expanding_mean
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0


## 4. Label Encoding

In [36]:
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder() 
category=['event_name_1','event_type_1','event_name_2','event_type_2','id','item_id','dept_id','cat_id','store_id','state_id']
for i in tqdm.tqdm(category):
    df_final[i+'_']=labelencoder.fit_transform(df_final[i])

100%|██████████| 10/10 [01:34<00:00,  9.49s/it]


In [41]:
## Dropping the columns that have been encoded
df_final=df_final.drop(['event_name_1','event_type_1','event_name_2','event_type_2','id','item_id','dept_id','cat_id','store_id','state_id'],axis = 1)

In [37]:
df_final=reduce(df_final)

In [42]:
df_final

,d,sold,date,wm_yr_wk,weekday,wday,month,year,snap_CA,snap_TX,...,event_name_1_,event_type_1_,event_name_2_,event_type_2_,id_,item_id_,dept_id_,cat_id_,store_id_,state_id_
0,d_1,0,2011-01-29,11101,Saturday,1,1,2011,0,0,...,19,2,3,1,14370,1437,3,1,0,0
1,d_1,0,2011-01-29,11101,Saturday,1,1,2011,0,0,...,19,2,3,1,14380,1438,3,1,0,0
2,d_1,0,2011-01-29,11101,Saturday,1,1,2011,0,0,...,19,2,3,1,14390,1439,3,1,0,0
3,d_1,0,2011-01-29,11101,Saturday,1,1,2011,0,0,...,19,2,3,1,14400,1440,3,1,0,0
4,d_1,0,2011-01-29,11101,Saturday,1,1,2011,0,0,...,19,2,3,1,14410,1441,3,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59181085,d_1941,1,2016-05-22,11617,Sunday,2,5,2016,0,0,...,19,2,3,1,14329,1432,2,0,9,2
59181086,d_1941,0,2016-05-22,11617,Sunday,2,5,2016,0,0,...,19,2,3,1,14339,1433,2,0,9,2
59181087,d_1941,2,2016-05-22,11617,Sunday,2,5,2016,0,0,...,19,2,3,1,14349,1434,2,0,9,2
59181088,d_1941,0,2016-05-22,11617,Sunday,2,5,2016,0,0,...,19,2,3,1,14359,1435,2,0,9,2


In [43]:
l=[]
for i in df_final['d']:
    l.append(i.split('_')[1])
df_final['day']=l
#https://stackoverflow.com/questions/15891038/change-column-type-in-pandas
df_final['day']=df_final['day'].astype(np.int16)

In [44]:
df_final_copy = df_final

In [34]:
df_final.columns

Index(['d', 'sold', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year',
       'snap_CA', 'snap_TX', 'snap_WI', 'is_high_sale_months',
       'is_high_sale_day', 'is_christmas', 'sell_price', 'lag_1', 'lag_7',
       'lag_14', 'lag_21', 'lag_28', 'lag_35', 'lag_42', 'lag_49', 'lag_56',
       'lag_63', 'lag_70', 'rolling_mean_7', 'rolling_mean_14',
       'rolling_mean_28', 'rolling_mean_35', 'rolling_mean_42',
       'expanding_mean', 'event_name_1_', 'event_type_1_', 'event_name_2_',
       'event_type_2_', 'id_', 'item_id_', 'dept_id_', 'cat_id_', 'store_id_',
       'state_id_', 'day'],
      dtype='object')

## A. Separating Target and independent Variables - Simple Features

In [45]:
data=df_final[['sold','wm_yr_wk','wday','month','year','snap_CA','snap_TX','snap_WI','event_name_1_','event_type_1_','event_name_2_','event_type_2_','id_','item_id_','dept_id_','cat_id_','store_id_','state_id_','day']]     

In [47]:
x=data[['wm_yr_wk','wday','month','year','snap_CA','snap_TX','snap_WI',	'event_name_1_','event_type_1_','event_name_2_','event_type_2_','id_','item_id_','dept_id_','cat_id_','store_id_','state_id_','day']]

In [48]:
y=data[['day','sold']]

In [49]:
x_train=x.loc[x['day']<1914]
x_test=x.loc[x['day']>=1914]

In [ ]:
print("x_train {}".format(x_train.shape))
print("\nx_test {}".format(x_test.shape))

In [50]:
y_train=y.loc[y['day']<1914]
y_test=y.loc[y['day']>=1914]

In [ ]:
print("y_train {}".format(y_train.shape))
print("\ny_test {}".format(y_test.shape))

In [51]:
y_train.drop(columns=['day'],inplace=True)
y_test.drop(columns=['day'],inplace=True)

C:\Users\Owner\AppData\Local\Temp\ipykernel_3180\1698220734.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_train.drop(columns=['day'],inplace=True)
C:\Users\Owner\AppData\Local\Temp\ipykernel_3180\1698220734.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_test.drop(columns=['day'],inplace=True)


In [52]:
print("y_train {}".format(y_train.shape))
print("\ny_test {}".format(y_test.shape))

y_train (58327370, 1)

y_test (853720, 1)


## A.1 Linear regression with lasso. (L1 regularization)

In [53]:
with open(r'simple_feature.pkl','wb') as f:
    pickle.dump([x_train,x_test,y_train,y_test], f)

In [8]:
with open(r'simple_feature.pkl', 'rb') as f:
    x_train,x_test,y_train,y_test= pickle.load(f)

In [9]:
clf = linear_model.Lasso(alpha=0.1)
clf.fit(x_train,y_train)

Lasso(alpha=0.1)

In [10]:
lasso_y_predicted=clf.predict(x_test)

In [32]:
lasso_y_predicted.shape

(853720,)

In [12]:
RMSE=mean_squared_error(y_test['sold'], lasso_y_predicted, squared=True)
print('RMSE using Linear regreation with L1 :',RMSE)

RMSE using Linear regreation with L1 : 13.05074764699362


In [13]:
l1_r2 = metrics.r2_score(y_test['sold'], lasso_y_predicted)
print("R2: ",l1_r2)

R2:  0.015349773106556608


##  A.2 DecisionTreeRegressor

In [18]:
clf=DecisionTreeRegressor(random_state=0)
clf.fit(x_train,y_train)

DecisionTreeRegressor(random_state=0)

In [19]:
decison_y_predicted=clf.predict(x_test)

In [20]:
RMSE=mean_squared_error(y_test['sold'], decison_y_predicted, squared=True)
print('RMSE using DecisionTreeRegressor:',RMSE)

RMSE using DecisionTreeRegressor: 8.768130066063815


In [21]:
decison_acc = metrics.accuracy_score(y_test['sold'], decison_y_predicted)
print("Accuracy: ",decison_acc)

Accuracy:  0.4246603101719533


In [22]:
decision_r2 = metrics.r2_score(y_test['sold'], decison_y_predicted)
print("R2: ",decision_r2)

R2:  0.3384638572051627


## A.3 LGBMRegressor

In [14]:
gbm_clf=LGBMRegressor(n_jobs=- 1)
gbm_clf.fit(x_train,y_train)

LGBMRegressor()

In [15]:
gbm_y_predicted=gbm_clf.predict(x_test)

In [16]:
RMSE=mean_squared_error(y_test['sold'], gbm_y_predicted, squared=True)
print('RMSE using LGBMRegressor:',RMSE)

RMSE using LGBMRegressor: 11.926172051591175


In [17]:
gbm_r2 = metrics.r2_score(y_test['sold'], gbm_y_predicted)
print("R2: ",gbm_r2)

R2:  0.10019653017544594


## B. Separating Target and independent Variables - Adding Day feature

In [54]:
data_2=df_final[['sold','is_high_sale_months','is_high_sale_day','is_christmas','wm_yr_wk','wday','month','year','snap_CA','snap_TX','snap_WI','event_name_1_','event_type_1_','event_name_2_','event_type_2_','id_','item_id_','dept_id_','cat_id_','store_id_','state_id_','day']]

In [55]:
x=data_2[['is_high_sale_months','is_high_sale_day','is_christmas','wm_yr_wk','wday','month','year','snap_CA','snap_TX','snap_WI','event_name_1_','event_type_1_','event_name_2_','event_type_2_','id_','item_id_','dept_id_','cat_id_','store_id_','state_id_','day']]

In [56]:
y=data_2[['day','sold']]

In [57]:
x_train=x.loc[x['day']<1914]
x_test=x.loc[x['day']>=1914]

In [58]:
y_train=y.loc[y['day']<1914]
y_test=y.loc[y['day']>=1914]

In [59]:
with open(r'days_feature.pkl','wb') as f:
    pickle.dump([x_train,x_test,y_train,y_test], f)

In [23]:
with open(r'days_feature.pkl', 'rb') as f:
    x_train,x_test,y_train,y_test= pickle.load(f)

In [24]:
y_train.drop(columns=['day'],inplace=True)
y_test.drop(columns=['day'],inplace=True)

## B.1 Lasso regression

In [25]:
linear_clf = linear_model.Lasso(alpha=0.1)
linear_clf.fit(x_train,y_train)

Lasso(alpha=0.1)

In [26]:
lasso_y_predicted=linear_clf.predict(x_test)

In [27]:
lasso_y_predicted.shape

(853720,)

In [28]:
y_test.shape

(853720, 1)

In [29]:
RMSE=mean_squared_error(y_test['sold'], lasso_y_predicted, squared=True)
print('RMSE using Linear regreation with L1:',RMSE)

RMSE using Linear regreation with L1: 13.05074764699362


In [30]:
l1_r2 = metrics.r2_score(y_test['sold'], lasso_y_predicted)
print("R2: ",l1_r2)

R2:  0.015349773106556608


## B.2 Decision Trees

In [37]:
decision_clf=DecisionTreeRegressor(random_state=0)
decision_clf.fit(x_train,y_train)

DecisionTreeRegressor(random_state=0)

In [38]:
decision_y_predicted=decision_clf.predict(x_test)

In [39]:
RMSE=mean_squared_error(y_test['sold'], decision_y_predicted, squared=True)
print('RMSE using Decision True:',RMSE)

RMSE using Decision True: 8.678277421168533


In [40]:
decision_r2 = metrics.r2_score(y_test['sold'], decision_y_predicted)
print("R2: ",decision_r2)

R2:  0.3452430417834115


## B.3 Light GBM

In [33]:
gbm_clf=LGBMRegressor(n_jobs=- 1)
gbm_clf.fit(x_train,y_train)

LGBMRegressor()

In [34]:
gbm_y_predicted=gbm_clf.predict(x_test)

In [35]:
RMSE=mean_squared_error(y_test['sold'], gbm_y_predicted, squared=True)
print('RMSE using LGBMRegressor:',RMSE)

RMSE using LGBMRegressor: 11.908841022390906


In [36]:
gbm_r2 = metrics.r2_score(y_test['sold'], gbm_y_predicted)
print("R2: ",gbm_r2)

R2:  0.10150411823828565


## C. Separating Target and independent Variables -  Adding Lags, Rolling mean and Expanding window features

In [34]:
data_final=df_final[['sold','expanding_mean','rolling_mean_7','rolling_mean_14','rolling_mean_28','rolling_mean_35','rolling_mean_42','lag_1','lag_7','lag_14',	'lag_21','lag_28','lag_35','lag_42','lag_49','lag_56','lag_63','lag_70','is_high_sale_months','is_high_sale_day','is_christmas','wm_yr_wk','wday','month','year','snap_CA','snap_TX','snap_WI','event_name_1_','event_type_1_','event_name_2_','event_type_2_','id_','item_id_','dept_id_','cat_id_','store_id_','state_id_','day']]

In [35]:
x=data_final[['rolling_mean_7','expanding_mean','rolling_mean_14','rolling_mean_28','rolling_mean_35','rolling_mean_42','lag_1','lag_7','lag_14','lag_21','lag_28','lag_35','lag_42','lag_49','lag_56','lag_63','lag_70','is_high_sale_months','is_high_sale_day','is_christmas','wm_yr_wk','wday','month','year','snap_CA','snap_TX','snap_WI','event_name_1_','event_type_1_','event_name_2_','event_type_2_','id_','item_id_','dept_id_','cat_id_','store_id_','state_id_','day']]

In [36]:
y=data_final[['day','sold']]

In [37]:
x_train=x.loc[x['day']<1914]
x_test=x.loc[x['day']>=1914]

In [38]:
y_train=y.loc[y['day']<1914]
y_test=y.loc[y['day']>=1914]

In [39]:
y_train.drop(columns=['day'],inplace=True)
y_test.drop(columns=['day'],inplace=True)

C:\Users\Owner\AppData\Local\Temp\ipykernel_1808\1698220734.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_train.drop(columns=['day'],inplace=True)
C:\Users\Owner\AppData\Local\Temp\ipykernel_1808\1698220734.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_test.drop(columns=['day'],inplace=True)


In [40]:
x_train=reduce(x_train)
y_train=reduce(y_train)

C:\Users\Owner\anaconda3\lib\site-packages\downcast.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[cols[i]] = df[cols[i]].astype(np.float16)
C:\Users\Owner\anaconda3\lib\site-packages\downcast.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[cols[i]] = df[cols[i]].astype(np.float16)
C:\Users\Owner\anaconda3\lib\site-packages\downcast.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

C:\Users\Owner\anaconda3\lib\site-packages\downcast.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[cols[i]] = df[cols[i]].astype(np.int8)
C:\Users\Owner\anaconda3\lib\site-packages\downcast.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[cols[i]] = df[cols[i]].astype(np.int8)
C:\Users\Owner\anaconda3\lib\site-packages\downcast.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

In [41]:
x_train = x_train.fillna(0)

In [43]:
# x_train=reduce(x_train)
# y_train=reduce(y_train)

C:\Users\Owner\anaconda3\lib\site-packages\downcast.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[cols[i]] = df[cols[i]].astype(np.int16)


In [44]:
y_train.shape

(58327370, 1)

In [44]:
with open(r'all_features.pkl','wb') as f:
    pickle.dump([x_train,x_test,y_train,y_test], f)

In [4]:
with open(r'all_features.pkl', 'rb') as f:
    x_train,x_test,y_train,y_test= pickle.load(f)

## C.1 Decision Tree

In [6]:
from sklearn.tree import DecisionTreeRegressor
decision_clf=DecisionTreeRegressor(random_state=0)
decision_clf.fit(x_train,y_train)

DecisionTreeRegressor(random_state=0)

In [7]:
decision_y_predicted=decision_clf.predict(x_test)

In [8]:
RMSE=mean_squared_error(y_test['sold'], decision_y_predicted, squared=True)
print('RMSE using Decision True:',RMSE)

RMSE using Decision True: 7.642573677552359


In [9]:
print("y_train {}".format(y_train.shape))
print("\ny_test {}".format(y_test.shape))

y_train (58327370, 1)

y_test (853720, 1)


In [10]:
decision_r2 = metrics.r2_score(y_test['sold'], decision_y_predicted)
print("R2: ",decision_r2)

R2:  0.42338461295852936


## C.2 Light GBM

In [36]:
import lightgbm
from lightgbm import LGBMRegressor
gbm_clf=LGBMRegressor(n_jobs=- 1)
gbm_clf.fit(x_train,y_train)

LGBMRegressor()

In [37]:
gbm_y_predicted=gbm_clf.predict(x_test)

In [39]:
RMSE=mean_squared_error(y_test['sold'], gbm_y_predicted, squared=True)
print('RMSE using LGBMRegressor:',RMSE)

RMSE using LGBMRegressor: 3.0474313818653593


In [42]:
gbm_r2 = metrics.r2_score(y_test['sold'], gbm_y_predicted)
print("R2: ",gbm_r2)

R2:  0.7700779999154179


## C.3 Linear regression with lasso (L1 regularization)

In [45]:
clf = linear_model.Lasso(alpha=0.1)
clf.fit(x_train,y_train)

Lasso(alpha=0.1)

In [48]:
lasso_y_predicted=clf.predict(x_test)

In [49]:
RMSE=mean_squared_error(y_test['sold'], y_predicted, squared=True)
print('RMSE using Linear regreation with L1 :',RMSE)

RMSE using Linear regreation with L1 : 3.3347616


In [51]:
l1_r2 = metrics.r2_score(y_test['sold'], lasso_y_predicted)
print("R2: ",l1_r2)

R2:  0.7483995535105836


## Results:

In [47]:
from prettytable import PrettyTable

# Specify the Column Names while initializing the Table
myTable = PrettyTable(["Feature INFO", "Model", "RMSE", "R2"])


# Add rows
myTable.add_row(["A. Adding Simple Features", "", "", ""])
myTable.add_row(["A.1", "Linear regression with L1 regularization", "13.05", "0.02"])
myTable.add_row(["A.2", "Decision Tree", "8.77", "0.34"])
myTable.add_row(["A.3", "LGBM Regressor", "11.93", "0.10"])
myTable.add_row(["", "", "", ""])
myTable.add_row(["B. Adding Day Features", "", "", ""])
myTable.add_row(["B.1", "Linear regression with L1 regularization", "13.05", "0.02"])
myTable.add_row(["B.2", "Decision Tree", "8.68", "0.35"])
myTable.add_row(["B.3", "LGBM Regressor", "11.91", "0.10"])
myTable.add_row(["", "", "", ""])
myTable.add_row(["C. Adding lags, rolling window and expanding window Features", "", "", ""])
myTable.add_row(["C.1", "Linear regression with L1 regularization", "3.33", "0.75"])
myTable.add_row(["C.2", "Decision Tree", "7.64", "0.42"])
myTable.add_row(["C.3", "LGBM Regressor", "3.05", "0.77"])

myTable.vertical_char = "|"
myTable.junction_char = "+"

print(myTable)


+--------------------------------------------------------------+------------------------------------------+-------+------+
|                         Feature INFO                         |                  Model                   |  RMSE |  R2  |
+--------------------------------------------------------------+------------------------------------------+-------+------+
|                  A. Adding Simple Features                   |                                          |       |      |
|                             A.1                              | Linear regression with L1 regularization | 13.05 | 0.02 |
|                             A.2                              |              Decision Tree               |  8.77 | 0.34 |
|                             A.3                              |              LGBM Regressor              | 11.93 | 0.10 |
|                                                              |                                          |       |      |
|               

## Conclusion:
1. It can be observed that the model gives the best result with the LGBM regresssion with the least RMSE score and highest R2 value when day features, lags, rolling window and expanding window Features are used as features. 
2. Additionally, it can be noticed that using the simple features i.e not adding lags, day features and rolling window and expanding, the model performance is the worst. Only adding the day features to the simple features doesnt help improve the model at all